# Implementação de um modelo baseado em Naive Bayes para inferir a probabilidade de uma frase ser spam ou não.

- Baseado na aula 9 - Naive Bayes e um detector de Spam
- Professor: Caio Gomes

In [1]:
import kagglehub # api do kaggle

# Download latest version
path = kagglehub.dataset_download("uciml/sms-spam-collection-dataset")

print("Path to dataset files:", path)

100%|██████████| 211k/211k [00:00<00:00, 35.6MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/uciml/sms-spam-collection-dataset/versions/1


In [2]:
import nltk
import pandas as pd
from collections import defaultdict
import numpy as np

In [22]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
# Caminho completo do arquivo CSV
file_path = f"{path}/spam.csv"  # ajuste o nome do arquivo conforme necessário

# Carrega o arquivo em um DataFrame
df = pd.read_csv(file_path, sep=',', encoding='ISO-8859-1')

In [14]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [54]:
# Renomeando as colunas
df = df.rename(columns={"v1": "classe", "v2": "mensagem", "Unnamed: 2": "dicionario_tolken"})

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   classe             5572 non-null   object
 1   mensagem           5572 non-null   object
 2   dicionario_tolken  50 non-null     object
 3   Unnamed: 3         12 non-null     object
 4   Unnamed: 4         6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [56]:
# verificando proporção de spam e não spam da base de dados
df["classe"].value_counts(normalize=True) * 100

# ham = não spam - 86%
# spam = spam - 14 %

,proportion
classe,
ham,86.593683
spam,13.406317


# Criação da classe que calcula o conforme o modelo de Naive Bayes

In [57]:
class NaiveClassifier2Classes(object):
  # criamos o objeto com a probabilidade prior da classe 1 e a lista de probabilidades
  def __init__(self, p_spam, probabilities):
    self.p_spam = p_spam # probabilidade de ser spam
    self.p_ham = 1 - p_spam # 1 - probabilidade de ser spam
    self.probabilities = probabilities

  # aqui criamos a função da classe que vai devolver o fit no modelo em escala log
  def fit(self, array_tokens) -> float:
    p = {"spam":0, "ham":0}
    for k1, i1 in self.probabilities.items():
      for w in array_tokens:
        p[k1] = p[k1] + self.probabilities[k1][w]

    return {"spam":p["spam"] + np.log(self.p_spam), "ham":p["ham"] + np.log(self.p_ham)}

In [58]:
# Prepando a base e separando em tokens

from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english", ignore_stopwords=True)

def word_prep(phrase, stemm=True):
  tokenized = tokenizer.tokenize(phrase.lower()) # deixando a frase com letras minusculas
  stem = []
  for token in tokenized:
    stem.append(stemmer.stem(token))
  if stemm == True:
    return stem
  return tokenized

# sem tokenization e stemming

In [59]:
# Sem usar a função word_prep
def return_value():
  def return_value2():
    return 1 # spam
  return defaultdict(return_value2)
contagem = defaultdict(return_value)
for index, row in df.iterrows():
  for word in tokenizer.tokenize(row["mensagem"]):
    contagem[row["classe"]][word.lower()] += 1

total = {"spam":0, "ham":0}
for k1, i1 in contagem.items():
  for k2, i2 in i1.items():
    total[k1] += i2

def return_value():
  def return_value2():
    return 0 # não é spam
  return defaultdict(return_value2)
probabilidade = defaultdict(return_value)

for k1, i1 in contagem.items():
  for k2, i2 in i1.items():
    probabilidade[k1][k2] += i2*1./total[k1]
prob_ln = defaultdict(return_value)

for k1, i1 in probabilidade.items():
  for k2, i2 in i1.items():
    prob_ln[k1][k2] += np.log(i2)

In [60]:
nc = NaiveClassifier2Classes(0.135, prob_ln)

In [61]:
phrase = "Hello, where are you?"

In [62]:
nc.fit(phrase) # adiciono a nova frase para ele ver se é spam

{'spam': -98.90137639965448, 'ham': -111.64027460880946}

# com stemming

- vamos repetir o código acima, mas agora em vez de treinar o modelo com palavras sem stemming, utilizando a função word_prep

In [63]:
# modifico para usar a função word_prep
def return_value():
  def return_value2():
    return 1 # spam
  return defaultdict(return_value2)
contagem = defaultdict(return_value)
for index, row in df.iterrows():
  for word in word_prep(row["mensagem"]): # muda aqui, word_prep é a função que pre processa a frase
    contagem[row["classe"]][word] += 1

total = {"spam":0, "ham":0}
for k1, i1 in contagem.items():
  for k2, i2 in i1.items():
    total[k1] += i2

def return_value():
  def return_value2():
    return 0 # não é spam
  return defaultdict(return_value2)
probabilidade = defaultdict(return_value)

for k1, i1 in contagem.items():
  for k2, i2 in i1.items():
    probabilidade[k1][k2] += i2*1./total[k1]
prob_ln = defaultdict(return_value)

for k1, i1 in probabilidade.items():
  for k2, i2 in i1.items():
    prob_ln[k1][k2] += np.log(i2)

In [64]:
nc2 = NaiveClassifier2Classes(0.135, prob_ln)

In [65]:
phrase = "You win free tickets for the WorldCup"

In [66]:
# não faz stemmização
nc.fit(phrase)

{'spam': -193.64407575962153, 'ham': -191.74840688857566}

In [67]:
# não faz stemmização
nc.fit(word_prep(phrase, stemm=False))

{'spam': -34.7221482892686, 'ham': -38.219299815541234}

In [68]:
# faz stemmização
nc2.fit(word_prep(phrase, stemm=True))

{'spam': -34.42422725600199, 'ham': -37.108343737329065}

#### Como resultado, essa frase seria classificada como spam.

In [69]:
import joblib

In [70]:
# salvando o modelo
joblib.dump(NaiveClassifier2Classes, "NaiveClassifier2Classes.pkl")

['NaiveClassifier2Classes.pkl']